In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

dataset = pd.read_csv('dataset/train/train.csv')

train_dataset, dev_dataset = train_test_split(dataset, stratify= dataset.label, test_size= 0.1, random_state=1004)

label = train_dataset['label']

print(train_dataset.shape)
print(train_dataset.head(5))

(29223, 6)
          id                                           sentence  \
22874  22874  KMH그룹(회장 최상주)이 운영하는 종합여행기업 보물섬투어는 뉴캐슬 지역의 ‘헌터밸...   
31527  31527  이상으로서 합법적인 입후보등록이 아닌 것은 대체로 알 수 있을 것이거니와 기자협회에...   
21296  21296  이후, 그는 2003년 9월 29일에 맨체스터 유나이티드 FC를 상대로 챔피언스리그...   
21877  21877  박능후 중앙재난안전대책본부 1차장(보건복지부 장관)은 24일 “(코로나19의)확산세...   
25327  25327  정치적으로 공납제를 기초로 한 왕정으로 호메로스에 나오는 그리스의 여러 왕들에 비하...   

                                          subject_entity  \
22874  {'word': 'KMH', 'start_idx': 0, 'end_idx': 2, ...   
31527  {'word': '장택상', 'start_idx': 56, 'end_idx': 58...   
21296  {'word': '맨체스터 유나이티드 FC', 'start_idx': 21, 'en...   
21877  {'word': '보건복지부', 'start_idx': 19, 'end_idx': ...   
25327  {'word': '호메로스', 'start_idx': 22, 'end_idx': 2...   

                                           object_entity  \
22874  {'word': '최상주', 'start_idx': 9, 'end_idx': 11,...   
31527  {'word': '경찰관', 'start_idx': 89, 'end_idx': 91...   
21296  {'word': '2003', 'start_idx': 7, 'end_

In [4]:
print(label.value_counts())
print(label.value_counts().index)

no_relation                            8581
org:top_members/employees              3856
per:employee_of                        3216
per:title                              1893
org:member_of                          1679
org:alternate_names                    1188
per:origin                             1111
org:place_of_headquarters              1075
per:date_of_birth                      1017
per:alternate_names                     901
per:spouse                              715
per:colleagues                          481
per:parents                             468
org:founded                             405
org:members                             378
per:date_of_death                       376
org:product                             342
per:children                            274
per:place_of_residence                  174
per:other_family                        171
per:place_of_birth                      149
org:founded_by                          140
per:product                     

In [5]:
replace_label = ['per:alternate_names', 'org:alternate_names', 'per:alternate_name', 'per:other_family', 'per:colleagues', 'per:siblings', 'per:spouse']
replace_member = ['org_members', 'org:menbers_of']
replace_family = ['per:parents', 'per:children']

id = len(train_dataset)
newData = []

for i in range(len(train_dataset)):
    if train_dataset.iloc[i, 4] in replace_label:
        data = train_dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        newData.append(data)
        id += 1
    elif train_dataset.iloc[i, 4] in replace_member:
        data = train_dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        if data['label'] == 'org_members':
            data['label'] = 'org:menbers_of'
        else: data['label'] = 'org_members'

        newData.append(data)
        id += 1
    elif train_dataset.iloc[i, 4] in replace_family:
        data = train_dataset.iloc[i, :]
        
        data['id'] = id
        tmp = data['subject_entity']
        data['subject_entity'] = data['object_entity']
        data['object_entity'] = tmp

        if data['label'] == 'per:parents': 
            data['label'] = 'per:children'
        else: data['label'] = 'per:parents'

        newData.append(data)
        id += 1


/tmp/ipykernel_4208/2914749360.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['id'] = id
/tmp/ipykernel_4208/2914749360.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['subject_entity'] = data['object_entity']
/tmp/ipykernel_4208/2914749360.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_entity'] = tmp
/tmp/ipykernel_4208/2914749360.py:36: SettingWithCopyWarning: 
A value is trying to be set on a 

In [6]:
selected_label = []
under_2000_name = [l for l in label.value_counts().index]
under_2000 = [l for l in label.value_counts()]

for i in range(len(under_2000)):
    if under_2000[i] < 2000:
        selected_label.append(under_2000_name[i])

print(selected_label)

['per:title', 'org:member_of', 'org:alternate_names', 'per:origin', 'org:place_of_headquarters', 'per:date_of_birth', 'per:alternate_names', 'per:spouse', 'per:colleagues', 'per:parents', 'org:founded', 'org:members', 'per:date_of_death', 'org:product', 'per:children', 'per:place_of_residence', 'per:other_family', 'per:place_of_birth', 'org:founded_by', 'per:product', 'per:siblings', 'org:political/religious_affiliation', 'per:religion', 'per:schools_attended', 'org:dissolved', 'org:number_of_employees/members', 'per:place_of_death']


In [7]:
import random
import re

def Random_Insertion(text, sub_word, ob_word, sub_length, ob_length):
    word_list = ['오늘', '내일', '어제', '머신', '러닝']
    new_word = word_list[random.randint(0, len(word_list) - 1)]

    text_len = len(text.split())
    text_word_list = text.split()

    sub_start = 0
    ob_start = 0

    for idx in range(len(text_word_list)):
        if re.search(sub_word , text_word_list[idx]):
            sub_start = idx
            break
    
    sub_end = sub_start + sub_length
    sub_index = [i for i in range(sub_start, sub_end)]

    for idx in range(len(text_word_list)):
        if re.search(ob_word , text_word_list[idx]):
            ob_start = idx
            break

    ob_end = ob_start + ob_length
    ob_index = [i for i in range(ob_start, ob_end)]

    valid_num = [i for i in range(0, text_len) if i not in sub_index and i not in ob_index]
    
    if valid_num != []:
        target_num = random.choice(valid_num)
        text_word_list.insert(target_num, new_word)
    else:
        target_num = random.randInt(0, text_len)
        text_word_list.insert(target_num, new_word)

    return ' '.join(text_word_list)


In [8]:
for idx in range(len(train_dataset)):
    if train_dataset.iloc[idx, 4] in selected_label:
        data = train_dataset.iloc[idx, :]

        subject_dict = {}
        subject_entity = eval(data['subject_entity'])
        subject_word = subject_entity['word']
        subject_word_list = subject_word.split()
        
        object_dict = {}
        object_entity = eval(data['object_entity'])
        object_word = object_entity['word']
        object_word_list = object_word.split()

        try:
            data['id'] = id
            data['sentence'] = Random_Insertion(data['sentence'], subject_word_list[0], object_word_list[0], len(subject_word.split()), len(object_word.split()))
            
            subject_dict = {'word': f'{subject_word}', 'start_idx': re.search(subject_word, data['sentence']).start(), 'end_idx': re.search(subject_word, data['sentence']).end()-1}
            object_dict = {'word': f'{object_word}', 'start_idx': re.search(object_word, data['sentence']).start(), 'end_idx': re.search(object_word, data['sentence']).end()-1}

            data['subject_entity'] = subject_dict
            data['object_entity'] = object_dict

            newData.append(data)
            id += 1
        except:
            continue

/tmp/ipykernel_4208/4268144360.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['id'] = id
/tmp/ipykernel_4208/4268144360.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentence'] = Random_Insertion(data['sentence'], subject_word_list[0], object_word_list[0], len(subject_word.split()), len(object_word.split()))
/tmp/ipykernel_4208/4268144360.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['subject_entity'

In [9]:
newDF = pd.DataFrame(newData, columns=['id', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'])
train_dataset = train_dataset.append(newDF, ignore_index=True)

print(newDF['label'].value_counts())
print(newDF.shape)

print(train_dataset['label'].value_counts())
print(train_dataset.shape)

print(dev_dataset['label'].value_counts())
print(dev_dataset.shape)

newDF.to_csv('dataset/train/addDataset.csv', index=False)
train_dataset.to_csv('dataset/train/train_dataset.csv', index=False)
dev_dataset.to_csv('dataset/train/dev_dataset.csv', index=False)

org:alternate_names                    2368
per:title                              1887
per:alternate_names                    1774
org:member_of                          1662
per:spouse                             1423
per:origin                             1108
org:place_of_headquarters              1069
per:date_of_birth                      1015
per:colleagues                          961
per:children                            738
per:parents                             732
org:founded                             403
org:members                             376
per:date_of_death                       369
org:product                             339
per:other_family                        339
per:siblings                            243
per:place_of_residence                  172
per:place_of_birth                      147
org:founded_by                          139
per:product                             124
org:political/religious_affiliation      87
per:religion                    